In [138]:
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from geopy.geocoders import Nominatim
import gc
import glob
import geopandas as gpd
import psycopg2
from psycopg2 import sql
from shapely.geometry import LineString, MultiLineString
from shapely import wkb
import geojson

In [139]:
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[mpl.font_manager.FontProperties(fname=font).get_name() for font in font_list if 'Yu' in font]

['Yu Gothic', 'Yu Gothic', 'Yu Gothic', 'Yu Gothic']

In [140]:
plt.rcParams['font.family'] = 'Yu Gothic'

In [ ]:
file_path = glob.glob("./*.zip")[0]
gdf = gpd.GeoDataFrame.from_file(file_path, encoding='CP949')
gdf = gdf.to_crs({"init": "epsg:4326"})

In [141]:
emd = gpd.read_file("./HangJeongDong_ver20230701.geojson", encoding="euc-kr")

In [142]:
emd['dong'] = emd['adm_nm'].str.split(' ').str[-1]

In [143]:
emd = emd.drop(columns=['temp'])

In [144]:
emd[emd["sggnm"] == "양천구"]

,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd8,geometry,dong
222,서울특별시 양천구 목1동,1115051,1147051000,11470,11,서울특별시,양천구,11150510,"MULTIPOLYGON (((126.88848 37.53072, 126.88761 ...",목1동
223,서울특별시 양천구 목2동,1115052,1147052000,11470,11,서울특별시,양천구,11150520,"MULTIPOLYGON (((126.88516 37.54359, 126.88349 ...",목2동
224,서울특별시 양천구 목3동,1115053,1147053000,11470,11,서울특별시,양천구,11150530,"MULTIPOLYGON (((126.86498 37.55093, 126.87070 ...",목3동
225,서울특별시 양천구 목4동,1115054,1147054000,11470,11,서울특별시,양천구,11150540,"MULTIPOLYGON (((126.87267 37.53757, 126.87254 ...",목4동
226,서울특별시 양천구 신월1동,1115057,1147056000,11470,11,서울특별시,양천구,11150570,"MULTIPOLYGON (((126.84049 37.52654, 126.83489 ...",신월1동
227,서울특별시 양천구 신월2동,1115058,1147057000,11470,11,서울특별시,양천구,11150580,"MULTIPOLYGON (((126.85161 37.52172, 126.84853 ...",신월2동
228,서울특별시 양천구 신월3동,1115059,1147058000,11470,11,서울특별시,양천구,11150590,"MULTIPOLYGON (((126.82851 37.53591, 126.82965 ...",신월3동
229,서울특별시 양천구 신월4동,1115060,1147059000,11470,11,서울특별시,양천구,11150600,"MULTIPOLYGON (((126.84444 37.52004, 126.83945 ...",신월4동
230,서울특별시 양천구 신월5동,1115061,1147060000,11470,11,서울특별시,양천구,11150610,"MULTIPOLYGON (((126.82992 37.54613, 126.83008 ...",신월5동
231,서울특별시 양천구 신월6동,1115062,1147061000,11470,11,서울특별시,양천구,11150620,"MULTIPOLYGON (((126.84848 37.51698, 126.84857 ...",신월6동


In [103]:
# CSV로 저장할 파일명
emd_file_name = "emd.csv"
# GeoDataFrame을 CSV 파일로 저장
emd.to_csv(f"./{emd_file_name}", encoding='utf8', index=False)

In [ ]:
# 메모리 정리
del emd
emd.collect()

## DB 설정 및 인서트

In [136]:
# Assuming you have a PostgreSQL connection already established
conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'postgres',
        user = 'postgres',
        password = 'vldpfzpdl12#',
        port = 15432
    )
cursor=conn.cursor()

In [117]:
# 테스트
cursor.execute(f"INSERT INTO yc_emd (adm_nm, sidonm, sggnm, dong, adm_cd, adm_cd2, geom) VALUES (%s, %s, %s, %s, %s, %s, %s)", 
               ['서울특별시 양천구 목1동', '서울특별시', '양천구', '목1동', 1115051, 1147051000, wkb.dumps('MULTIPOLYGON (((126.88848 37.53072, 126.88761 37.52948, 126.88705 37.52878, 126.88696 37.52866, 126.88659 37.52831, 126.88631 37.52806, 126.88603 37.52793, 126.88535 37.52761, 126.88476 37.52740, 126.88173 37.52632, 126.88137 37.52600, 126.88051 37.52523, 126.88049 37.52515, 126.88028 37.52436, 126.87982 37.52220, 126.87445 37.52267, 126.87116 37.52313, 126.86859 37.52396, 126.86540 37.52257, 126.86442 37.52583, 126.86432 37.52621, 126.86421 37.52716, 126.86417 37.52751, 126.86405 37.52924, 126.86401 37.52980, 126.86719 37.53022, 126.87037 37.53063, 126.87393 37.53111, 126.87505 37.53126, 126.87961 37.53176, 126.88060 37.53184, 126.88083 37.53186, 126.88148 37.53189, 126.88253 37.53187, 126.88259 37.53187, 126.88295 37.53184, 126.88374 37.53178, 126.88507 37.53156, 126.88643 37.53132, 126.88651 37.53129, 126.88848 37.53072)))', hex=True)])
conn.commit()

TypeError: One of the arguments is of incorrect type. Please provide only Geometry objects.

In [137]:
for _, row in emd.iterrows():
    query = "INSERT INTO yc_emd (adm_nm, sidonm, sggnm, dong, adm_cd, adm_cd2, geom) VALUES ('{0}', '{1}', '{2}', '{3}', {4}, {5}, '{6}')".format(row[0], row[5], row[6], row[9], row[1], row[2], wkb.dumps(row[8], hex=True))
    cursor.execute(query)
conn.commit()

In [116]:
filtered_emd = emd[emd["dong"] == "목1동"]
print(filtered_emd.to_string())

            adm_nm   adm_cd     adm_cd2    sgg sido sidonm sggnm   adm_cd8                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               geometry dong
222  서울특별시 양천구 목1동  1115051  1147051000  11470   11  서울특별시   양천구  11150510  MULTI

In [89]:
len(emd)

3523

In [79]:
type(emd)

geopandas.geodataframe.GeoDataFrame

In [80]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [ ]:
with open("./emd_20230729/emd.shp") as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [7]:
gdf.tail()

NameError: name 'gdf' is not defined

In [5]:
# CSV로 저장할 파일명
new_file_name = "yc_road.csv"
# GeoDataFrame을 CSV 파일로 저장
gdf.to_csv(f"./{new_file_name}", encoding='utf8', index=False)
# 메모리 정리
del gdf
gc.collect()

0

In [ ]:
def process_geometry(geometry):
    print(geometry)
    coord_string = geometry.wkt[len('LINESTRING ('):-1]
    coordinates = [tuple(map(float, coord.split())) for coord in coord_string.split(',')]
    
    # Create a Shapely LineString from the extracted coordinates
    linestring = LineString(coordinates)
    
    # Convert the LineString to a MultiLineString
    multilinestring = MultiLineString([linestring])
    return multilinestring

In [ ]:
linestring_data = 'LINESTRING (126.90295533441757 37.4846860726053, 126.90391625932587 37.48464013534261, 126.90449167202657 37.48462285051611, 126.90563802372202 37.48460247760803, 126.90643867133988 37.48466710464008, 126.90706293500978 37.484823874887404, 126.90872425175941 37.48530255944304, 126.91014217832195 37.4856979063246, 126.91133473556538 37.48605156428905, 126.91197675291336 37.48630067465691, 126.91253017488867 37.48662541690091, 126.91302493404527 37.4869276867154, 126.91326371484111 37.48708959488873, 126.91357331232896 37.48729950654804, 126.91653461046394 37.48911972094082, 126.91703215476596 37.489477882924064, 126.91765407165218 37.489898110928806, 126.91768588541841 37.4899026745456, 126.9185230043999 37.48994144990997, 126.9186201634896 37.48994595103764, 126.91983870951296 37.489989326896946, 126.92097442452098 37.49001721764318, 126.92132117827427 37.49002575172057, 126.92236619600149 37.49004481878315, 126.92290542269083 37.49005465730263, 126.92360025096282 37.489954926349405, 126.92383373905932 37.48996336929478, 126.9241655637569 37.48996426274743, 126.92425517429476 37.48996450157906, 126.92430865793418 37.48996464466069, 126.92540823610246 37.489903216100934, 126.9254587189009 37.48989650011823, 126.92596881633975 37.48982861266382, 126.92597426737858 37.489827715149026, 126.92614795499279 37.48979827326757, 126.9264940220396 37.48973551360686, 126.92678251422848 37.48969289893107, 126.92680128994785 37.489687297574854, 126.92705216063993 37.48961224946881, 126.92733525652582 37.4895372841357, 126.92795811685572 37.489366096090315, 126.92801127532982 37.48935146990026, 126.92805667799566 37.489335829744206, 126.92839483337045 37.489219350968064, 126.92866892464693 37.48909907456882, 126.9289090564006 37.488993688221, 126.92916122341828 37.48888302889225, 126.92922692338355 37.488854183039116, 126.92922762472394 37.48885387712817, 126.92984656706956 37.488571682478174, 126.92987693996857 37.48855784351309, 126.93020516815362 37.48841042217624, 126.93039337615933 37.488325895645865, 126.93048314755178 37.48828556681386, 126.9305127053822 37.48827249304486, 126.93062727159828 37.488221790466284, 126.93067670424517 37.48819991679508, 126.93094081212708 37.488083016876786, 126.93158804717086 37.48779652317549, 126.93168375535058 37.48775417869951, 126.9321114157877 37.48756407631649, 126.93211984295996 37.487560333104376, 126.93217859572937 37.48753409442331, 126.93253202781092 37.487376157694705, 126.93272382727919 37.487290467241074, 126.93309268599512 37.487125645154315, 126.93347510557597 37.48695474796678, 126.93347892885909 37.486953038267316, 126.93416359196381 37.486667452940665, 126.93419889139403 37.486655498758914, 126.93420490848158 37.48665344790545, 126.9345328048045 37.4865422705686, 126.93488624995383 37.486422439102924, 126.93490136041945 37.486417311919176, 126.93499229697453 37.486371042708456, 126.93532857684444 37.48619996934995, 126.9355598249315 37.486082319848514, 126.9357201805558 37.48601441973724, 126.93580674329428 37.48597776113835, 126.93613874653472 37.485831568723874, 126.93652284705189 37.485662446540296, 126.93662490674058 37.48561703813496, 126.93675520996189 37.48555906715323, 126.93690793141263 37.48549112519849, 126.93708557070343 37.4854120963054, 126.93726656788111 37.48533285272977, 126.93755265398129 37.485207552587184, 126.93756583112373 37.48520179331421, 126.93759433443 37.48518937492565, 126.9383281436195 37.48485702098741, 126.93838610066649 37.48483075175017, 126.93869886884602 37.4846891099057, 126.93870545178038 37.48468610406207, 126.93881987223186 37.48463429420545, 126.93892232482524 37.484588929065104, 126.9389656335741 37.48456975166918, 126.9390557912475 37.48452983996912, 126.93918066134368 37.48447454841113, 126.93923360666085 37.484451105255296, 126.93950617103822 37.484330406358296, 126.9397142300807 37.484238306920936, 126.9397336391132 37.48422969449127, 126.93993408558339 37.48414095141825, 126.93994660643278 37.484135371736706, 126.94028743979403 37.48398343283659, 126.9406137038185 37.48383799066818, 126.94082628487521 37.48374320661129, 126.94114850897348 37.483599571896605, 126.94115030734213 37.48359876191435, 126.94135208556719 37.48350881877375, 126.9415571140517 37.4834398972935, 126.94166733552653 37.48340282291528, 126.94177245634123 37.483367484787344, 126.94191079604842 37.48333099202597, 126.94199118978273 37.48330975090534, 126.94214553111719 37.483269013203135, 126.9422691108074 37.483253019391455, 126.94230461638274 37.483248388023014, 126.94258158215086 37.483212495452555, 126.94285574340365 37.48317696124076, 126.94292575930159 37.48316788685731, 126.9430384382818 37.48315327437676, 126.94319874425999 37.48313249542613, 126.94354358782147 37.48308779525917, 126.94356939079461 37.483085267092626, 126.94362038608357 37.48308035460358, 126.94387064749633 37.48305605081826, 126.94394187125464 37.48304916585468, 126.94401533382167 37.48304205668736, 126.9443588110669 37.4830087241109, 126.94443410530636 37.4830014443789, 126.9447184458407 37.482973902641405, 126.94483704635428 37.48296237271548, 126.94503771391375 37.48294291707461, 126.94504021281486 37.48294263895866, 126.94536740769357 37.48290983626667, 126.94593475445467 37.48285283686369, 126.9462192981496 37.4828242644675, 126.94640135399203 37.48280597811032, 126.94733216328459 37.48271560570322, 126.94738880032371 37.48271010852345, 126.94748676462791 37.48270060274508, 126.94751828870419 37.48269751771098, 126.94767951589111 37.482681886827386, 126.94786507577992 37.482663870111146, 126.94787094416276 37.4826632600947, 126.94803574460096 37.48264675637581, 126.94811984648591 37.48263836103896, 126.94844793192723 37.48260550516644, 126.94864748997936 37.48258549328425, 126.94927425007286 37.482522747011664, 126.94928116999426 37.482522020259864, 126.94977789888571 37.48247168361904, 126.9500971523568 37.482439350889095, 126.950684460459 37.48237983257419, 126.95068457352718 37.482379823612845, 126.95101476186659 37.48234370890069, 126.95183818475373 37.48225365148146, 126.95197325832044 37.482238850680055, 126.9522731087281 37.48220608046138, 126.95261456860786 37.48216873160204, 126.95262353546983 37.48216710450016, 126.95290279753183 37.48211569145294, 126.95317628208801 37.482046219313176, 126.9534510100111 37.48197644070475, 126.95387938350272 37.48182452513626, 126.95392750078523 37.48180922750018, 126.95397873918282 37.48179291296846, 126.95399144981447 37.48178889060488, 126.9541184769877 37.481748549731314, 126.95419658398976 37.4817237404257, 126.95431906249856 37.48167081052365, 126.95473824303845 37.481406226930275, 126.9547454704547 37.481401670723514, 126.95478756820613 37.4813750537739, 126.9550651050058 37.481199521962, 126.95512424672016 37.48116319879355, 126.95551412368091 37.48092372083934, 126.95561750805342 37.48084591450379, 126.95569773027051 37.4807430512523, 126.95575960377805 37.4805086345984, 126.95577740350915 37.48042755133891, 126.95579016497908 37.48033955544305, 126.9558254520633 37.480095929219466, 126.9558557625367 37.47989244969092, 126.95588353016564 37.479705709778884, 126.95599638574707 37.479331296950484, 126.95605759275863 37.47926050176714, 126.95614728110726 37.479204584020934, 126.95679244592058 37.47895283775986, 126.95691702836395 37.47890642918826, 126.95704192730311 37.47885986743363, 126.95718978165488 37.47880476365155, 126.9571947911691 37.47880290045662, 126.95753152879809 37.478671083555604, 126.95794036639391 37.478510982819714, 126.95818050564377 37.478419403189335, 126.95818310650732 37.478418413040956, 126.95831846447702 37.47836674508636, 126.9583239828263 37.478364638758876, 126.95842589132752 37.478325698617894, 126.95848671360991 37.478309223434856, 126.95850436437966 37.478304463560306, 126.95861878462716 37.47827142035935, 126.95864895293884 37.478262691587815, 126.95873853934722 37.47824129822518, 126.95902915822722 37.47812609283534, 126.95937907548527 37.477987842235585, 126.95954177571582 37.47792355958323, 126.960399192736 37.47758478691734, 126.96071500836355 37.47746601816317, 126.96074702070078 37.4774539738034, 126.96075447258559 37.47745106613719, 126.96076583699356 37.47744662811621, 126.96095811581101 37.47737149662763, 126.96198304264546 37.476970276294445, 126.96259973035517 37.47672884120539, 126.96316244010453 37.47651416316831, 126.96320177900807 37.476499156221514, 126.96320725191242 37.476496950540295, 126.96382657009423 37.47624722065822, 126.9643013134901 37.47606408021238, 126.9644307497631 37.47601361071135, 126.96491918440385 37.47582315593155, 126.96520191360906 37.475712915407705, 126.96547473678073 37.47560653648475, 126.96558560152374 37.47557034994769, 126.965620765759 37.47555968375535)'
# Extract the coordinates from the Linestring data
# Extract the coordinates from the Linestring data after removing 'LINESTRING (' and ')'
coord_string = linestring_data[len('LINESTRING ('):-1]
coordinates = [tuple(map(float, coord.split())) for coord in coord_string.split(',')]

# Create a Shapely LineString from the extracted coordinates
linestring = LineString(coordinates)

# Convert the LineString to a MultiLineString
multilinestring = MultiLineString([linestring])

In [28]:
# Assuming you have a PostgreSQL connection already established
conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'postgres',
        user = 'postgres',
        password = 'vldpfzpdl12#',
        port = 15432
    )
cursor=conn.cursor()

In [ ]:
# Insert the MultiLineString into the database
insert_query = sql.SQL("INSERT INTO yc_road (geom) VALUES (%s);")
cursor.execute(insert_query, [wkb.dumps(multilinestring, hex=True)])

# Commit the transaction and close the connection
conn.commit()
conn.close()

In [10]:
gdf.tail()

,ALWNC_DE,ALWNC_RESN,BSI_INT,ENG_RN,MVMN_DE,MVMN_RESN,MVM_RES_CD,NTFC_DE,OPERT_DE,RBP_CN,...,RDS_MAN_NO,REP_CN,RN,RN_CD,ROAD_BT,ROAD_LT,ROA_CLS_SE,SIG_CD,WDR_RD_CD,geometry
67036,20230120,삼양로173길로 되어 있는 도로명으로 인해 도선사 및 북한산 방문자들이 불편을 느끼...,10,Doseonsa-gil,20230120,삼양로173길로 되어 있는 도로명으로 인해 도선사 및 북한산 방문자들이 불편을 느끼...,01,20230120,20230120153124,우이동 산68-1,...,4208,우이동 289,도선사길,4862319,12.000,1390.011,4,11305,3,"LINESTRING (127.00202 37.65805, 127.00197 37.6..."
67037,20230208,수서역세권 공공주택지구 조성사업,20,Jagok-ro 11-gil,20230324,수서역세권 공공주택지구 조성사업,01,20230324,20230324090747,자곡동 100-17,...,7319,자곡동 194-9,자곡로11길,4862412,20.000,750.100,4,11680,3,"LINESTRING (127.10854 37.47627, 127.10798 37.4..."
67038,20230324,수서역세권 공공주택지구 조성사업,20,Jagok-ro 13-gil,20230324,수서역세권 공공주택지구 조성사업,01,20230324,20230324090654,자곡동 123,...,7320,수서동 224-3,자곡로13길,4862411,20.000,1110.120,4,11680,3,"LINESTRING (127.11089 37.47703, 127.11050 37.4..."
67039,20230324,수서역세권 공공주택지구 조성사업,20,Bamgogae-ro 17-gil,20230324,수서역세권 공공주택지구 조성사업,01,20230324,20230324090837,자곡동 323-1,...,7321,자곡동 148-2,밤고개로17길,4862410,20.000,644.940,4,11680,3,"LINESTRING (127.10532 37.47960, 127.10631 37.4..."
67040,20230329,고가도로 신규생성,20,Bongwongogado-ro,20230329,고가도로 신규생성,01,20230329,20230731174900,대신동 143-14,...,4044,신촌동 1-142,봉원고가도로,3354099,14.782,532.200,3,11410,3,"LINESTRING (126.94485 37.56400, 126.94512 37.5..."


In [49]:
gdf2 = [
    {'geom': 'LINESTRING (127.05807031415185 37.61516946298505)'},
    {'geom': 'POINT (127.05807031415185 37.61516946298505)'},
    {'geom': 'LINESTRING (127.05807031415185 37.61516946298505)'},
]

filtered_gdf = [item for item in gdf if item['geometry'].startswith('LINESTRING')]


# Print the filtered list
for item in filtered_gdf:
    print(item)

TypeError: string indices must be integers, not 'str'

In [56]:
for _, row in gdf.iterrows():
    cursor.execute(f"INSERT INTO yc_road (eng_rn, rn, geom) VALUES (%s, %s, %s)", [row[3], row[13], wkb.dumps(row[20], hex=True)])
conn.commit()

In [27]:
conn.close()

In [40]:
print(type(gdf))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [55]:
for _, row in gdf.iterrows():
    geom_wkt = row['geometry'].wkt
    if not geom_wkt.startswith('LINESTRING'):
        print(row[3], row[13], row[20])

Yongin Seoul Expressway 용인서울고속도로 MULTILINESTRING ((127.09151736169352 37.46567500435493, 127.09138550050471 37.46568424674274, 127.09123118582522 37.465692948126325, 127.09091906936919 37.46567153207817, 127.09086886600323 37.46565895599554, 127.09075455038628 37.46563021205702, 127.0906520997072 37.46559713243941, 127.09055868541265 37.46556035361136, 127.09047079964698 37.465514829009976, 127.09039603098232 37.46547235789085, 127.09031411909073 37.46541223422068, 127.09028288609296 37.46538270476244, 127.09021362930513 37.46531725379127, 127.09017128586677 37.46526268473722, 127.09003688456013 37.46507285389264, 127.0899461054754 37.464876232726546, 127.08992876215021 37.46483570053122, 127.08991762932631 37.46478966756415, 127.08991102263737 37.464748586731446, 127.0899050640407 37.46467903189001, 127.08988470519687 37.46438702996053, 127.08987951234793 37.46429864536141, 127.08986454907007 37.463978976751896, 127.08982835963498 37.46329727714225), (127.09525915692586 37.45639489316

In [ ]:
for _, row in gdf.iterrows():
    geom_wkt = row['geometry'].wkt
    if not geom_wkt.startswith('LINESTRING'):
        print(row[3], row[13])
        merged_line = row['geometry'].unary_union
        print(merged_line)

In [61]:
decimal_number = 37.53440432466932

# Convert the number to a string
number_str = str(decimal_number)

# Split the string at the decimal point
parts = number_str.split('.')

# Check the length of the decimal part
if len(parts) > 1 and len(parts[1]) > 10:
    print("The number has more than 10 decimal places.")
    formatted_number = format(decimal_number, '.10f')
    print(formatted_number)
else:
    print("The number has 10 or fewer decimal places.")




The number has more than 10 decimal places.
37.5344043247


In [3]:
a = "KMFZCY7JACU812366/2023/09/18/20230918_132101_meta.json"
b = a.split('/')[0]
b

'KMFZCY7JACU812366'